In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
random_seed = 42

In [14]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]

In [15]:
from sklearn.model_selection import train_test_split
def preprocess_train(_data, is_remove_outliers):
    data = _data.copy()
    data = data.drop("match_id", axis = 1)
    if is_remove_outliers:
        data = delete_outliers(data)
    data = data.dropna()
    return split_train(data)

def split_train(_data):
    raw_train, raw_test = train_test_split(_data, test_size=0.2, random_state=random_seed)
    Y_train = pd.DataFrame(raw_train['target'])
    X_train = raw_train.drop('target', axis = 1)
    Y_test = pd.DataFrame(raw_test['target'])
    X_test = raw_test.drop('target', axis = 1)
    Y_train["target"] = Y_train['target'].astype(int)
    Y_test["target"] = Y_test['target'].astype(int)
    return X_train,Y_train, X_test, Y_test

def delete_outliers(_data):
    data = _data.copy()
    data = data.where(data["game_time"] < 10000000)
    return data

In [16]:
X_train,Y_train, X_test, Y_test = preprocess_train(train, True)

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
model = SVC(kernel='rbf', gamma= 10, C=10)
model.fit(X_train, Y_train)

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AttributeError: 'SVC' object has no attribute 'best_params_'

In [ ]:
from sklearn.metrics import roc_auc_score
print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, model.predict(X_test))}")

Score : 0.5
ROC-AUC score : 0.5


In [ ]:
from write import write
write(model)